In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import requests
import time
import hmac
import hashlib
import base64
import os  # 파일 존재 여부 확인을 위해 import os
from collections import Counter  # 빈출 단어 계산을 위해 import Counter
from openpyxl import load_workbook # 엑셀 파일 수정을 위해 import

# Signature 클래스: API 서명 생성
class Signature:
    @staticmethod
    def generate(timestamp, method, uri, secret_key):
        message = "{}.{}.{}".format(timestamp, method, uri)
        hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)
        return base64.b64encode(hash.digest()).decode("utf-8")

# API 요청을 위한 헤더를 가져오는 함수
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = Signature.generate(timestamp, method, uri, secret_key)
    return {
        'Content-Type': 'application/json; charset=UTF-8',
        'X-Timestamp': timestamp,
        'X-API-KEY': api_key,
        'X-Customer': str(customer_id),
        'X-Signature': signature
    }

# 네이버 API에서 연관 키워드 결과를 가져오는 함수
def get_results(hintKeywords):
    BASE_URL = 'https://api.naver.com'
    API_KEY = '010000000013d50306951198c56725e6d082916ed4870b354963d7e7119eb766144d8c58c0'  # 실제 API_KEY로 변경하세요
    SECRET_KEY = 'AQAAAAAT1QMGlRGYxWcl5tCCkW7UFW5GoTbNMAkwsweVpmyjWw=='  # 실제 SECRET_KEY로 변경하세요
    CUSTOMER_ID = '2496501'  # 실제 CUSTOMER_ID로 변경하세요

    uri = '/keywordstool'
    method = 'GET'

    params = {
        'hintKeywords': hintKeywords,
        'showDetail': '1'
    }

    r = requests.get(BASE_URL + uri, params=params,
                     headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

    if r.status_code == 200:
        return pd.DataFrame(r.json()['keywordList'])
    else:
        messagebox.showerror("Error", f"API Error: {r.status_code}, {r.text}")
        return None

# 연관 키워드로 결과 리스트박스 업데이트 함수
def update_related_keywords():
    hintKeywords = keyword_entry.get()  # 키워드 입력 필드에서 입력 가져오기
    if not hintKeywords:
        messagebox.showwarning("입력 에러", "키워드를 입력하세요.")
        return
    results_df = get_results(hintKeywords)

    if results_df is not None:
        # API 응답의 컬럼 이름 확인
        print("API 응답 컬럼:", results_df.columns)

        # 이제 'relKeyword'를 정확한 컬럼 이름으로 사용
        if 'relKeyword' in results_df.columns:
            related_keywords_listbox.delete(0, tk.END)  # 이전 결과 지우기
            for keyword in results_df['relKeyword']:
                related_keywords_listbox.insert(tk.END, keyword)
        else:
            messagebox.showerror("에러", "API 응답에 예상되는 'relKeyword' 컬럼이 없습니다.")

# 제외 단어 엑셀 파일 시트 이름 가져오는 함수
def get_excel_sheet_names(filename="제외단어.xlsx"):
    try:
        if os.path.exists(filename):
            xls = pd.ExcelFile(filename)
            sheet_names = xls.sheet_names
            xls.close() # 명시적으로 닫기
            return sheet_names
        else:
            return [] # 파일 없으면 빈 리스트 반환
    except Exception as e:
        messagebox.showerror("시트 목록 에러", f"시트 목록을 불러오는 중 오류 발생: {e}")
        return []

# 제외 단어 엑셀 시트 리스트박스 업데이트 함수
def update_excel_sheet_listbox():
    sheet_names = get_excel_sheet_names()
    excel_sheet_listbox.delete(0, tk.END) # 기존 목록 삭제
    for name in sheet_names:
        excel_sheet_listbox.insert(tk.END, name)
    if not sheet_names:
        excel_sheet_listbox.insert(tk.END, "제외단어.xlsx 파일 없음 또는 시트 없음") # 파일 없을 때 메시지 표시

# 제외 단어를 엑셀 파일에 저장하는 함수 (선택된 시트에 저장, 다른 시트 유지)
def save_excluded_words():
    excluded_words_input = exclude_entry.get()  # 제외 단어 입력 필드에서 입력 가져오기
    selected_sheet_index = excel_sheet_listbox.curselection() # 선택된 시트 인덱스 가져오기
    if not selected_sheet_index:
        messagebox.showwarning("시트 선택 에러", "제외 단어를 저장할 엑셀 시트를 선택하세요.")
        return
    selected_sheet_name = excel_sheet_listbox.get(selected_sheet_index[0]) # 선택된 시트 이름 가져오기

    if excluded_words_input:
        excluded_words_list_new = [word.strip() for word in excluded_words_input.split()] # 띄어쓰기로 구분, strip()으로 공백 제거
        excluded_words_existing_in_sheet = load_excluded_words_from_excel(sheet_name=selected_sheet_name) # 선택한 시트의 기존 제외 단어만 로드 # 변경
        combined_excluded_words = list(set(excluded_words_existing_in_sheet + excluded_words_list_new)) # 선택한 시트의 기존 단어 + 새로운 제외 단어 합치고 중복 제거 # 변경
        df = pd.DataFrame({'제외 단어': combined_excluded_words}) # 컬럼 이름을 '제외 단어'로 설정

        try:
            with pd.ExcelWriter("제외단어.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: # mode='a' (append), sheet_name 지정
                df.to_excel(writer, sheet_name=selected_sheet_name, index=False) # 엑셀 파일로 저장, index 제거, 시트 이름 지정
            messagebox.showinfo("저장됨", f"제외 단어가 엑셀 파일 '{selected_sheet_name}' 시트에 저장되었습니다 (제외단어.xlsx).")
            update_excel_sheet_listbox() # 저장 후 시트 목록 업데이트
        except Exception as e:
            messagebox.showerror("저장 에러", f"엑셀 파일 저장 중 오류 발생: {e}")
        exclude_entry.delete(0, tk.END)  # 제외 단어 입력 필드 지우기
    else:
        messagebox.showwarning("입력 에러", "제외할 단어를 입력하세요.")

# 제외 단어 엑셀 파일에서 불러오는 함수 (특정 시트 또는 모든 시트에서 불러오기) # 수정
def load_excluded_words_from_excel(sheet_name=None): # sheet_name 파라미터 추가, None이면 모든 시트에서 로드 # 수정
    excluded_words_excel = []
    if os.path.exists("제외단어.xlsx"):
        try:
            xls = pd.ExcelFile("제외단어.xlsx") # ExcelFile 객체 생성
            sheet_names = xls.sheet_names # 모든 시트 이름 가져오기
            sheets_to_load = [sheet_name] if sheet_name else sheet_names # 특정 시트 이름이 지정되면 해당 시트만, 아니면 모든 시트 # 수정

            for current_sheet_name in sheets_to_load: # 로드할 시트 목록 순회 # 수정
                if current_sheet_name in sheet_names: # 시트가 엑셀 파일에 존재하는지 확인 (에러 방지) # 추가
                    df_excel = pd.read_excel(xls, sheet_name=current_sheet_name) # 각 시트별로 데이터프레임 로드
                    if '제외 단어' in df_excel.columns:
                        excluded_words_excel.extend([str(word).strip() for word in df_excel['제외 단어'].tolist() if pd.notna(word)]) # 엑셀 파일에서 제외 단어 목록 로드 및 공백 제거, NaN 값 처리
            xls.close() # ExcelFile 객체 닫기
        except Exception as e:
            messagebox.showerror("불러오기 에러", f"제외 단어 엑셀 파일 불러오는 중 오류 발생: {e}")
    return excluded_words_excel

# 주 키워드 엑셀 파일에 저장하는 함수
def save_main_keywords_to_excel():
    main_keywords_input = main_keywords_entry_for_save.get() # 키워드 모음 단어 입력 필드에서 입력 가져오기
    if main_keywords_input:
        main_keywords_list_new = [word.strip() for word in main_keywords_input.split()] # 띄어쓰기로 구분, strip()으로 공백 제거
        main_keywords_existing = load_main_keywords_from_excel() # 기존 주 키워드 로드
        combined_main_keywords = list(set(main_keywords_existing + main_keywords_list_new)) # 기존 + 새로운 주 키워드 합치고 중복 제거
        df = pd.DataFrame({'주키워드': combined_main_keywords}) # 컬럼 이름을 '주키워드'로 설정
        try:
            df.to_excel("키워드모음.xlsx", index=False) # 엑셀 파일로 저장, index 제거
            messagebox.showinfo("저장됨", "키워드 모음 단어가 엑셀 파일에 저장되었습니다 (키워드모음.xlsx).")
        except Exception as e:
            messagebox.showerror("저장 에러", f"키워드 모음 엑셀 파일 저장 중 오류 발생: {e}")
        main_keywords_entry_for_save.delete(0, tk.END) # 키워드 모음 단어 입력 필드 지우기
    else:
        messagebox.showwarning("입력 에러", "키워드 모음에 저장할 단어를 입력하세요.")


# 주 키워드 엑셀 파일에서 불러오는 함수
def load_main_keywords_from_excel():
    main_keywords_excel = []
    if os.path.exists("키워드모음.xlsx"):
        try:
            df_excel = pd.read_excel("키워드모음.xlsx")
            if '주키워드' in df_excel.columns: # 컬럼명 "주 키워드" 로 변경
                main_keywords_excel = [str(word).strip() for word in df_excel['주키워드'].tolist() if pd.notna(word)] # 엑셀 파일에서 주 키워드 목록 로드 및 공백 제거, NaN 값 처리
        except Exception as e:
            messagebox.showerror("불러오기 에러", f"주 키워드 엑셀 파일 불러오는 중 오류 발생: {e}")
    return main_keywords_excel


# 세키워드 기반으로 주키워드 업데이트 함수 (입력 키워드 제거, 분리, 중복 제거 및 표시)
def update_main_keywords_from_detailed(): # 함수명 변경: 상세 키워드 -> 세키워드
    search_term = keyword_entry.get().strip() # "키워드 입력" 창의 내용을 검색어로 사용 및 공백 제거
    main_keywords_excel_list = load_main_keywords_from_excel() # 엑셀 파일에서 주키워드 리스트 로드 # 메시지 변경: 주 키워드 -> 주키워드
    excluded_words_excel_list = load_excluded_words_from_excel() # 👈👈👈 제외단어 엑셀 시트 로드
    all_main_keyword_parts = [] # 분리된 주키워드 파트들을 저장할 리스트 초기화 # 메시지 변경: 주 키워드 -> 주키워드

    if not search_term:
        messagebox.showwarning("입력 에러", "주키워드 추출을 위해 키워드를 입력하세요.") # 메시지 변경: 주 키워드 -> 주키워드
        return

    main_keywords_listbox.delete(0, tk.END) # 이전 결과 지우기

    for detailed_keyword in detailed_keywords_listbox.get(0, tk.END):  # 세키워드 리스트의 모든 키워드에 대해

        # 1. 입력된 키워드 제거
        keyword_removed = detailed_keyword.lower().replace(search_term.lower(), "").strip()  # 입력 키워드 제거 (대소문자 무시) 및 공백 제거

        # 2. 키워드모음.XLSX 단어 기준으로 분리
        current_keyword_parts = [keyword_removed]  # 분리되지 않은 경우 자기 자신을 요소로 갖는 리스트로 시작
        split_parts = []
        temp_parts = [keyword_removed]

        extracted_main_keywords = [] # 👈👈👈 추출된 주키워드 저장 리스트 초기화

        for main_keyword in main_keywords_excel_list:  # 엑셀 파일의 주 키워드 목록 순회
            next_parts = []
            for p in temp_parts:
                if main_keyword.strip() and main_keyword.lower() in p.lower():  # 주 키워드가 비어있지 않고, 현재 파트에 주 키워드가 포함되면
                    parts = p.lower().split(main_keyword.lower()) # split() 대신 split() 유지 (일단 분리된 텍스트 얻기 위해)
                    if parts: # 분리된 파트가 있는 경우 (split() 결과가 빈 리스트가 아닐 때만)
                        extracted_main_keywords.append(main_keyword) # 👈👈👈 추출된 주키워드 리스트에 추가
                        next_parts.extend([part.strip() for part in parts if part.strip()]) # 분리된 텍스트 파트 추가 (공백 제거)
                        #print(f"    Extracted main keyword '{main_keyword}' from part '{p}'") # Debug: 추출된 주키워드 출력
                    else:
                        next_parts.append(p) # 분리 실패 시 원래 파트 유지
                    #print(f"    Splitting part '{p}' by main_keyword '{main_keyword}' - parts: {parts}") # Debug: Splitting action and parts

                else:
                    next_parts.append(p)  # 분리할 주 키워드 없으면 현재 파트 그대로 유지
            temp_parts = [p.strip() for p in next_parts if p.strip()]
        split_parts.extend(temp_parts)
        all_main_keyword_parts.extend(split_parts + extracted_main_keywords) # 👈👈👈 분리된 텍스트 파트 + 추출된 주키워드 모두 추가
        #print(f"  Split parts for detailed_keyword: {split_parts}")
        #print(f"  Extracted main keywords for detailed_keyword: {extracted_main_keywords}") # Debug: 추출된 주키워드들 출력

    unique_main_keyword_parts = sorted(list(set(all_main_keyword_parts)), key=lambda x: x)  # 중복 제거 및 정렬 (가나다 순)

    filtered_main_keyword_parts = [] # 👈👈👈 제외 단어 필터링된 주키워드 파트 리스트 초기화
    for part in unique_main_keyword_parts:
        is_excluded = False
        for excluded_word in excluded_words_excel_list: # 👈👈👈 제외단어 엑셀 시트 검사
            if excluded_word.strip() and excluded_word.lower() == part.lower(): # 👈👈👈 제외 단어와 일치하는지 확인 (대소문자 무시)
                is_excluded = True
                break # 일치하는 제외 단어 발견 시 루프 종료
        if not is_excluded: # 제외 단어가 아니면 필터링된 리스트에 추가
            filtered_main_keyword_parts.append(part)

    for part in filtered_main_keyword_parts:  # 필터링된 파트들을 주키워드 리스트박스에 삽입
        if len(part) > 1:  # ✨✨✨ 한글자 주키워드 제외 ✨✨✨
            main_keywords_listbox.insert(tk.END, part)

    if main_keywords_listbox.size() == 0:
        messagebox.showinfo("일치하는 항목 없음", "추출된 주키워드가 없습니다.")  # 메시지 변경: 주 키워드 -> 주키워드

# 주키워드 리스트박스에서 선택된 키워드를 삭제하는 함수
def delete_main_keyword(event):
    selected_indices = main_keywords_listbox.curselection() # 선택된 항목들의 인덱스
    if selected_indices: # 선택된 항목이 있을 경우에만 삭제
        # reversed를 사용하여 인덱스 꼬임 문제 방지
        for index in reversed(selected_indices):
            main_keywords_listbox.delete(index)
    


# 주키워드 리스트박스에서 선택된 키워드를 제외 단어 입력 필드에 삽입하는 함수
def insert_excluded_words_from_listbox():
    selected_indices = main_keywords_listbox.curselection() # 선택된 항목들의 인덱스
    selected_keywords = [main_keywords_listbox.get(index) for index in selected_indices] # 선택된 인덱스에 해당하는 키워드 리스트
    joined_keywords = " ".join(selected_keywords) # 키워드들을 공백으로 연결
    exclude_entry.delete(0, tk.END) # 기존 내용 삭제
    exclude_entry.insert(0, joined_keywords) # 새 내용 삽입

# 주키워드 리스트박스에서 선택된 키워드를 키워드모음 단어 입력 필드에 삽입하는 함수 (새로 추가)
def insert_main_keywords_from_listbox():
    selected_indices = main_keywords_listbox.curselection()
    selected_keywords = [main_keywords_listbox.get(index) for index in selected_indices]
    joined_keywords = " ".join(selected_keywords)
    main_keywords_entry_for_save.delete(0, tk.END) # 기존 내용 삭제
    main_keywords_entry_for_save.insert(0, joined_keywords) # 새 내용 삽입


# 메인 키워드 리스트박스 업데이트 함수 (직접 입력)
def update_main_keywords():
    pass # 주키워드 직접 추가 기능 제거

# 입력을 기반으로 세키워드 업데이트 함수 (중복 제거 및 가나다 순 정렬 추가)
def update_detailed_keywords(): # 함수명 변경: 상세 키워드 -> 세키워드
    search_term = keyword_entry.get() # "키워드 입력" 창의 내용을 검색어로 사용
    excluded_words_input = exclude_entry.get() # 제외 단어 입력 필드 내용 가져오기
    excluded_words_entry_list = [word.strip() for word in excluded_words_input.split(',')] if excluded_words_input else [] # 제외 단어 입력 창에서 제외 단어 리스트 생성 (쉼표 구분 유지)
    excluded_words_excel_list = load_excluded_words_from_excel(sheet_name="공통") # 엑셀 파일에서 제외 단어 리스트 로드
    combined_excluded_words = list(set(excluded_words_entry_list + excluded_words_excel_list)) # 입력 창과 엑셀 파일의 제외 단어 합치기 (중복 제거)

    if not search_term:
        messagebox.showwarning("입력 에러", "세키워드 검색을 위해 키워드를 입력하세요.") # 메시지 변경: 상세 키워드 -> 세키워드
        return

    detailed_keywords_listbox.delete(0, tk.END)  # 이전 결과 지우기
    detailed_keywords = [] # 세키워드를 저장할 리스트 초기화 # 메시지 변경: 상세 키워드 -> 세키워드

    for keyword in related_keywords_listbox.get(0, tk.END):  # 모든 연관 키워드 반복
        if search_term.lower() in keyword.lower(): # 1. 검색어가 포함된 연관 검색어 (대소문자 구분 없이 비교)
            if keyword.lower().endswith(search_term.lower()): # 2. 검색어가 마지막에 위치하는 연관 검색어 (대소문자 구분 없이 비교)
                modified_keyword = keyword # 수정할 키워드 초기화
                for excluded_word in combined_excluded_words: # 3. 합쳐진 제외 단어 목록으로 처리
                    if excluded_word.strip() and excluded_word.lower() in modified_keyword.lower(): # 제외 단어가 비어있지 않고, 키워드에 포함되어 있다면 (대소문자 구분 없이 비교)
                        modified_keyword_parts = modified_keyword.lower().split(excluded_word.lower()) # 제외 단어로 분리
                        modified_keyword = "".join(modified_keyword_parts).strip() # 제외 단어 제거 후 공백 제거 (단순 분리 후 재결합으로 제외 단어 제거)
                detailed_keywords.append(modified_keyword) # 세키워드 리스트에 추가 # 메시지 변경: 상세 키워드 -> 세키워드

    unique_detailed_keywords = sorted(list(set(detailed_keywords)), key=lambda x: x) # 중복 제거 후 정렬 (가나다 순)

    for keyword in unique_detailed_keywords: # 정렬된 유니크한 키워드들을 리스트박스에 삽입
        detailed_keywords_listbox.insert(tk.END, keyword)

    if detailed_keywords_listbox.size() == 0:
        messagebox.showinfo("일치하는 항목 없음", f"'{search_term}'을(를) 포함하고, 마지막에 위치하는 키워드가 없습니다.") # 메시지 변경




# 메인 윈도우 생성
root = tk.Tk()
root.title("키워드 도구")
root.geometry("1200x900") # 윈도우 높임

# --- Column 0 (Leftmost) ---
keyword_label = tk.Label(root, text="키워드 입력:")
keyword_label.grid(row=0, column=0, padx=5, pady=5, sticky='w')
keyword_entry = tk.Entry(root, width=30)
keyword_entry.grid(row=1, column=0, padx=5, pady=5, sticky='ew')
search_button = tk.Button(root, text="연관 키워드 검색", command=update_related_keywords)
search_button.grid(row=2, column=0, padx=5, pady=5, sticky='ew')

add_detailed_keywords_button = tk.Button(root, text="세키워드 추출하기", command=update_detailed_keywords) # "상세 키워드 추가" -> "세키워드"
add_detailed_keywords_button.grid(row=3, column=0, padx=5, pady=5, sticky='ew') # row 위치 조정
add_main_keywords_from_detailed_button = tk.Button(root, text="주키워드 추출하기", command=update_main_keywords_from_detailed) # "주 키워드 추가" -> "주키워드"
add_main_keywords_from_detailed_button.grid(row=4, column=0, padx=5, pady=5, sticky='ew') # row 위치 조정

exclude_label = tk.Label(root, text="제외 단어 입력:") # "제외 단어 입력" 라벨을 제일 위로 이동
exclude_label.grid(row=5, column=0, padx=5, pady=5, sticky='w')
exclude_entry = tk.Entry(root, width=30) # "제외 단어 입력" 엔트리를 라벨 바로 아래로 이동
exclude_entry.grid(row=6, column=0, padx=5, pady=5, sticky='ew')
exclude_words_input_button = tk.Button(root, text="제외 단어 입력", command=insert_excluded_words_from_listbox) # "제외 단어 입력" 버튼을 엔트리 바로 아래로 이동
exclude_words_input_button.grid(row=7, column=0, padx=5, pady=5, sticky='ew')

excel_sheet_label = tk.Label(root, text="제외단어 엑셀 시트:") # 엑셀 시트 라벨을 그 다음으로 이동
excel_sheet_label.grid(row=8, column=0, padx=5, pady=5, sticky='w')
excel_sheet_listbox = tk.Listbox(root, width=30, height=5) # 엑셀 시트 리스트박스를 라벨 바로 아래로 이동
excel_sheet_listbox.grid(row=9, column=0, padx=5, pady=5, sticky='ew')
update_excel_sheet_listbox() # 시트 목록 초기화

save_exclude_button = tk.Button(root, text="제외 단어 저장", command=save_excluded_words) # "제외 단어 저장" 버튼을 마지막으로 이동
save_exclude_button.grid(row=10, column=0, padx=5, pady=5, sticky='ew')


main_keywords_label_for_save = tk.Label(root, text="키워드모음 단어 입력:")
main_keywords_label_for_save.grid(row=11, column=0, padx=5, pady=5, sticky='w')
main_keywords_entry_for_save = tk.Entry(root, width=30)
main_keywords_entry_for_save.grid(row=12, column=0, padx=5, pady=5, sticky='ew')
main_keywords_input_button = tk.Button(root, text="키워드모음 단어 입력", command=insert_main_keywords_from_listbox)
main_keywords_input_button.grid(row=13, column=0, padx=5, pady=5, sticky='ew')
save_main_keywords_button = tk.Button(root, text="키워드모음 단어 저장", command=save_main_keywords_to_excel)
save_main_keywords_button.grid(row=14, column=0, padx=5, pady=5, sticky='ew')



# --- Column 1 (Center-Left) ---
related_keywords_label = tk.Label(root, text="연관 키워드:")
related_keywords_label.grid(row=0, column=1, padx=5, pady=5, sticky='w')
related_keywords_listbox = tk.Listbox(root, width=30, height=20, selectmode=tk.EXTENDED)
related_keywords_listbox.grid(row=1, column=1, rowspan=14, padx=5, pady=5, sticky='nsew') # rowspan 증가


# --- Column 2 (Center-Right) ---
detailed_keywords_label = tk.Label(root, text="세키워드:") # "상세 키워드:" -> "세키워드:"
detailed_keywords_label.grid(row=0, column=2, padx=5, pady=5, sticky='w')
detailed_keywords_listbox = tk.Listbox(root, width=30, height=20, selectmode=tk.EXTENDED)
detailed_keywords_listbox.grid(row=1, column=2, rowspan=14, padx=5, pady=5, sticky='nsew') # rowspan 증가

# --- Column 3 (Rightmost) ---
main_keywords_label = tk.Label(root, text="주키워드:") # "메인 키워드:" -> "주키워드:"
main_keywords_label.grid(row=0, column=3, padx=5, pady=5, sticky='w')
main_keywords_listbox = tk.Listbox(root, width=30, height=20, selectmode=tk.EXTENDED) # 높이 증가
main_keywords_listbox.grid(row=1, column=3, rowspan=14, padx=5, pady=5, sticky='nsew') # rowspan 유지 (높이 증가로 충분)

# Delete 키 이벤트 바인딩
main_keywords_listbox.bind("<Delete>", delete_main_keyword)

# Enter 키 이벤트 바인딩 for keyword_entry
keyword_entry.bind("<Return>", lambda event: update_related_keywords())

# Configure column and row weights for resizing
root.columnconfigure(1, weight=1) # 연관 키워드 column expands
root.columnconfigure(2, weight=1) # 세키워드 column expands # 메시지 변경: 상세 키워드 -> 세키워드
root.columnconfigure(3, weight=1) # 주키워드 column expands # 메시지 변경: 주키워드 -> 주키워드
root.rowconfigure(1, weight=1)    # Make row 1 (where listboxes are) expandable


print("테스트 1")

# Tkinter 이벤트 루프 실행
root.mainloop()

테스트 1
